In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
import sys
from tqdm import tqdm
import time

from requests.exceptions import ConnectionError
from requests.packages.urllib3.exceptions import MaxRetryError
from requests.packages.urllib3.exceptions import ProxyError as urllib3_ProxyError

In [ ]:
drive.mount("/content/gdrive")
os.chdir('/content/gdrive/My Drive/github/4학년2학기과제및논문/졸업논문')

Mounted at /content/gdrive


In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 8.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic I

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
total_frame = pd.DataFrame()
for i in tqdm(range(1,295)):
  try:
    driver.get("http://www.aitimes.kr/news/articleList.html?page={}&total=5870&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type=sm".format(i))
    time.sleep(5)
    urls = driver.find_elements_by_css_selector("#user-container > div.float-center.max-width-1080 > div.user-content > section > article > div.article-list > section > div > div.list-titles > a")
    part_url = list(map(lambda x: x.get_attribute("href"),urls))
    part_name = list(map(lambda x: x.text,urls))
    part_frame = pd.DataFrame({"name":part_name,"url":part_url})
    total_frame = pd.concat([total_frame,part_frame])
    total_frame = total_frame.reset_index()
    total_frame = total_frame.drop(columns="index")
    total_frame.to_csv("./data/aitimes_news.csv")
  except ConnectionError as ce:
       if (isinstance(ce.args[0], MaxRetryError) and
           isinstance(ce.args[0].reason, urllib3_ProxyError)):
          # oops, requests should have handled this, but didn't.
          # see https://github.com/kennethreitz/requests/issues/3050
          pass



100%|██████████| 294/294 [38:44<00:00,  7.91s/it]


In [ ]:
total_frame = pd.read_csv("./data/aitimes_news.csv",index_col=[0])
total_frame

,name,url
0,아리랑 위성영상 이용한 AI 학습 데이터셋과 응용 AI 모델 공개한다,http://www.aitimes.kr/news/articleView.html?id...
1,"안랩, AI EXPO KOREA 2020에서 AI기반 보안 솔루션 소개",http://www.aitimes.kr/news/articleView.html?id...
2,"건국대 조용범 교수, 내 손안에서 AIoT를... 디바이스에 딥러닝 모듈 '맞춤 A...",http://www.aitimes.kr/news/articleView.html?id...
3,"씨브이티, 얼굴인식 스탠드얼론시스템 'Face A+' 선보인다",http://www.aitimes.kr/news/articleView.html?id...
4,"인공지능 기반 '스마트 팩토리' 구현... 심플랫폼, 'Pinpoint' 서비스로 ...",http://www.aitimes.kr/news/articleView.html?id...
...,...,...
5865,"인텔, 기가비트급 속도와 초저지연, 세계 최초 글로벌 5G 모뎀 발표",http://www.aitimes.kr/news/articleView.html?id...
5866,"포드-도요타, 앱과 자동차를 연결하는 'Smart Device Link (SDL)'...",http://www.aitimes.kr/news/articleView.html?id...
5867,"인텔, 7세대 인텔®코어™ 프로세서 및 모바일 워크스테이션용 인텔®제온®프로세서 출시",http://www.aitimes.kr/news/articleView.html?id...
5868,"SK-SM, 인공지능 서비스 나만의 비서 '에이브릴(SM-Aibril)' 공개",http://www.aitimes.kr/news/articleView.html?id...


In [ ]:
aitimes = pd.DataFrame()
for i in tqdm(range(0,len(total_frame))):
  driver.implicitly_wait(5)
  driver.get(total_frame["url"][i])
  title = total_frame["name"][i]
  content = driver.find_elements_by_css_selector("#article-view-content-div > p ")
  content = " ".join([j.text for j in content])
  classification = driver.find_elements_by_css_selector("#user-container > div.float-center.max-width-1080 > header > nav")
  classification = " ".join([k.text for k in classification])
  frame = pd.DataFrame.from_dict([{"name": title , "content" : content, "classification" : classification}])
  aitimes = aitimes.append(frame)
  aitimes.to_csv("./data/aitimes_content.csv")

100%|██████████| 5870/5870 [4:41:29<00:00,  2.88s/it]
